In [1]:
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the spaCy model for sentence segmentation
nlp = spacy.load("en_core_web_sm")

# Semantic splitting based on sentence boundaries and similarity
def semantic_splitting(text, threshold=0.3):
    # Parse the document
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]  # Extract sentences

    # Vectorize the sentences for similarity checking
    vectorizer = TfidfVectorizer().fit_transform(sentences)
    vectors = vectorizer.toarray()

    # Calculate pairwise cosine similarity between sentences
    similarities = cosine_similarity(vectors)

    # Initialize chunks with the first sentence
    chunks = [[sentences[0]]]

    # Group sentences into chunks based on similarity threshold
    for i in range(1, len(sentences)):
        sim_score = similarities[i-1, i]

        if sim_score >= threshold:
            # If the similarity is above the threshold, add to the current chunk
            chunks[-1].append(sentences[i])
        else:
            # Start a new chunk
            chunks.append([sentences[i]])

    # Join the sentences in each chunk to form coherent paragraphs
    return [' '.join(chunk) for chunk in chunks]

# Example usage
text = """
Long-context LLMs. There has long been efforts for enabling LLMs to handle long contexts 
(Guo et al., 2022; Beltagy et al., 2020; Chen et al., 
2023b). While recent LLMs like Gemini-1.5 (Reid
et al., 2024), GPT-4 (Achiam et al., 2023), Claude3 (Anthropic, 2024) achieve significantly larger
context window size, long-context prompting is
still expensive due to the quadratic computation
cost of transformers regarding to the input token
numbers. Recent work proposes methods to reduce
cost by prompt compression (Jiang et al., 2023),
model distillation (Hsieh et al., 2023), or LLM cascading (Chen et al., 2023a).

Retrieval-augmented generation. Augmenting
LLMs with relevant information retrieved from
various sources (Lewis et al., 2020), i.e., RAG,
has been successful in complementing LLMs with
external knowledge. RAG achieves good performance on various of tasks like language modeling
(Khandelwal et al., 2019; Shi et al., 2023) and QA
(Guu et al., 2020; Izacard and Grave, 2020), with
a significantly lower computation cost (Borgeaud
et al., 2022). Related to but different from our work,
recently works augment RAG with correction (Yan
et al., 2024), critique (Asai et al., 2023), or verification (Li et al., 2023) to improve retrieval quality
on knowledge-intensive tasks.
Long-context evaluation. Evaluating long-context
models is challenging due to the difficulty in
collecting and analyzing long texts. Recent researchers propose both synthetic tests like needlein-a-haystack (Greg Kamradt, 2023), Ruler (Hsieh
et al., 2024), or Counting Stars (Song et al., 2024),
and real datasets including LongBench (Bai et al.,
2023), ∞Bench (Zhang et al., 2024), L-Eval (An
et al., 2023), and others (Shaham et al., 2022; Yuan
et al., 2024; Maharana et al., 2024). Evaluating
on these datasets, recent works study the performance degradation over various context lengths
(Levy et al., 2024; Hsieh et al., 2024), the lostin-the-middle phenomenon (Liu et al., 2024), and
explore solutions (Kuratov et al., 2024). Related
to our work, Xu et al. (2023) compare RAG and
long-context prompting and find that long-context
models still lags behind RAG. This is different
from our findings, possibly due to consideration of
stronger LLMs and longer contexts in our work.
"""

# Perform semantic splitting
semantic_chunks = semantic_splitting(text)

# Print the chunks
for idx, chunk in enumerate(semantic_chunks):
    print(f"Chunk {idx+1}:\n{chunk}\n")


Chunk 1:

Long-context LLMs.

Chunk 2:
There has long been efforts for enabling LLMs to handle long contexts 
(Guo et al., 2022; Beltagy et al., 2020; Chen et al., 
2023b).

Chunk 3:
While recent LLMs like Gemini-1.5 (Reid
et al., 2024), GPT-4 (Achiam et al., 2023), Claude3 (Anthropic, 2024) achieve significantly larger
context window size, long-context prompting is
still expensive due to the quadratic computation
cost of transformers regarding to the input token
numbers.

Chunk 4:
Recent work proposes methods to reduce
cost by prompt compression (Jiang et al., 2023),
model distillation (Hsieh et al., 2023), or LLM cascading (Chen et al., 2023a).



Chunk 5:
Retrieval-augmented generation.

Chunk 6:
Augmenting
LLMs with relevant information retrieved from
various sources (Lewis et al., 2020), i.e., RAG,
has been successful in complementing LLMs with
external knowledge.

Chunk 7:
RAG achieves good performance on various of tasks like language modeling
(Khandelwal et al., 2019; Shi et al

In [2]:
from langchain_core.prompts import ChatPromptTemplate
import uuid
from langchain.chat_models import ChatOpenAI
import os
from typing import Optional
from langchain_core.pydantic_v1 import BaseModel
from langchain.chains import create_extraction_chain_pydantic
from dotenv import load_dotenv
from google.colab import userdata

load_dotenv()

class AgenticChunker:
    def __init__(self, openai_api_key=None):
        self.chunks = {}
        self.id_truncate_limit = 5

        # Whether or not to update/refine summaries and titles as you get new information
        self.generate_new_metadata_ind = True
        self.print_logging = True

        if openai_api_key is None:
            openai_api_key = userdata.get("open_ai_key")

        if openai_api_key is None:
            raise ValueError("API key is not provided and not found in environment variables")

        self.llm = ChatOpenAI(model='gpt-4-1106-preview', openai_api_key=openai_api_key, temperature=0)

    # Người dùng cung cấp danh sách các propositions (mệnh đề) để thêm vào hệ thống.
    def add_propositions(self, propositions):
        for proposition in propositions:
            self.add_proposition(proposition)

    def add_proposition(self, proposition):
        if self.print_logging:
            print (f"\nAdding: '{proposition}'")

        # If it's your first chunk, just make a new chunk and don't check for others
        if len(self.chunks) == 0:
            if self.print_logging:
                print ("No chunks, creating a new one")
            self._create_new_chunk(proposition)
            return

        chunk_id = self._find_relevant_chunk(proposition)

        # If a chunk was found then add the proposition to it
        if chunk_id:
            if self.print_logging:
                print (f"Chunk Found ({self.chunks[chunk_id]['chunk_id']}), adding to: {self.chunks[chunk_id]['title']}")
            self.add_proposition_to_chunk(chunk_id, proposition)
            return
        else:
            if self.print_logging:
                print ("No chunks found")
            # If a chunk wasn't found, then create a new one
            self._create_new_chunk(proposition)


    def add_proposition_to_chunk(self, chunk_id, proposition):
        # Add then
        self.chunks[chunk_id]['propositions'].append(proposition)

        # Then grab a new summary
        if self.generate_new_metadata_ind:
            self.chunks[chunk_id]['summary'] = self._update_chunk_summary(self.chunks[chunk_id])
            self.chunks[chunk_id]['title'] = self._update_chunk_title(self.chunks[chunk_id])


    # Cập nhật summary khi thêm Chunk mới
    def _update_chunk_summary(self, chunk):
        """
        If you add a new proposition to a chunk, you may want to update the summary or else they could get stale
        """
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic
                    A new proposition was just added to one of your chunks, you should generate a very brief 1-sentence summary which will inform viewers what a chunk group is about.

                    A good summary will say what the chunk is about, and give any clarifying instructions on what to add to the chunk.

                    You will be given a group of propositions which are in the chunk and the chunks current summary.

                    Your summaries should anticipate generalization. If you get a proposition about apples, generalize it to food.
                    Or month, generalize it to "date and times".

                    Example:
                    Input: Proposition: Greg likes to eat pizza
                    Output: This chunk contains information about the types of food Greg likes to eat.

                    Only respond with the chunk new summary, nothing else.
                    """,
                ),
                ("user", "Chunk's propositions:\n{proposition}\n\nCurrent chunk summary:\n{current_summary}"),
            ]
        )

        runnable = PROMPT | self.llm

        new_chunk_summary = runnable.invoke({
            "proposition": "\n".join(chunk['propositions']),
            "current_summary" : chunk['summary']
        }).content

        return new_chunk_summary

    def _update_chunk_title(self, chunk):
        """
        If you add a new proposition to a chunk, you may want to update the title or else it can get stale
        """
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic
                    A new proposition was just added to one of your chunks, you should generate a very brief updated chunk title which will inform viewers what a chunk group is about.

                    A good title will say what the chunk is about.

                    You will be given a group of propositions which are in the chunk, chunk summary and the chunk title.

                    Your title should anticipate generalization. If you get a proposition about apples, generalize it to food.
                    Or month, generalize it to "date and times".

                    Example:
                    Input: Summary: This chunk is about dates and times that the author talks about
                    Output: Date & Times

                    Only respond with the new chunk title, nothing else.
                    """,
                ),
                ("user", "Chunk's propositions:\n{proposition}\n\nChunk summary:\n{current_summary}\n\nCurrent chunk title:\n{current_title}"),
            ]
        )

        runnable = PROMPT | self.llm

        updated_chunk_title = runnable.invoke({
            "proposition": "\n".join(chunk['propositions']),
            "current_summary" : chunk['summary'],
            "current_title" : chunk['title']
        }).content

        return updated_chunk_title

    def _get_new_chunk_summary(self, proposition):
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic
                    You should generate a very brief 1-sentence summary which will inform viewers what a chunk group is about.

                    A good summary will say what the chunk is about, and give any clarifying instructions on what to add to the chunk.

                    You will be given a proposition which will go into a new chunk. This new chunk needs a summary.

                    Your summaries should anticipate generalization. If you get a proposition about apples, generalize it to food.
                    Or month, generalize it to "date and times".

                    Example:
                    Input: Proposition: Greg likes to eat pizza
                    Output: This chunk contains information about the types of food Greg likes to eat.

                    Only respond with the new chunk summary, nothing else.
                    """,
                ),
                ("user", "Determine the summary of the new chunk that this proposition will go into:\n{proposition}"),
            ]
        )

        runnable = PROMPT | self.llm

        new_chunk_summary = runnable.invoke({
            "proposition": proposition
        }).content

        return new_chunk_summary

    def _get_new_chunk_title(self, summary):
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic
                    You should generate a very brief few word chunk title which will inform viewers what a chunk group is about.

                    A good chunk title is brief but encompasses what the chunk is about

                    You will be given a summary of a chunk which needs a title

                    Your titles should anticipate generalization. If you get a proposition about apples, generalize it to food.
                    Or month, generalize it to "date and times".

                    Example:
                    Input: Summary: This chunk is about dates and times that the author talks about
                    Output: Date & Times

                    Only respond with the new chunk title, nothing else.
                    """,
                ),
                ("user", "Determine the title of the chunk that this summary belongs to:\n{summary}"),
            ]
        )

        runnable = PROMPT | self.llm

        new_chunk_title = runnable.invoke({
            "summary": summary
        }).content

        return new_chunk_title


    def _create_new_chunk(self, proposition):
        new_chunk_id = str(uuid.uuid4())[:self.id_truncate_limit] # I don't want long ids
        new_chunk_summary = self._get_new_chunk_summary(proposition)
        new_chunk_title = self._get_new_chunk_title(new_chunk_summary)

        self.chunks[new_chunk_id] = {
            'chunk_id' : new_chunk_id,
            'propositions': [proposition],
            'title' : new_chunk_title,
            'summary': new_chunk_summary,
            'chunk_index' : len(self.chunks)
        }
        if self.print_logging:
            print (f"Created new chunk ({new_chunk_id}): {new_chunk_title}")

    def get_chunk_outline(self):
        """
        Get a string which represents the chunks you currently have.
        This will be empty when you first start off
        """
        chunk_outline = ""

        for chunk_id, chunk in self.chunks.items():
            single_chunk_string = f"""Chunk ID: {chunk['chunk_id']}\nChunk Name: {chunk['title']}\nChunk Summary: {chunk['summary']}\n\n"""

            chunk_outline += single_chunk_string

        return chunk_outline

    # Kiểm tra xem có chunk nào hiện có phù hợp với proposition mới hay không
    # bằng cách so sánh ý nghĩa của chúng.
    def _find_relevant_chunk(self, proposition):
        current_chunk_outline = self.get_chunk_outline()

        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    Determine whether or not the "Proposition" should belong to any of the existing chunks.

                    A proposition should belong to a chunk of their meaning, direction, or intention are similar.
                    The goal is to group similar propositions and chunks.

                    If you think a proposition should be joined with a chunk, return the chunk id.
                    If you do not think an item should be joined with an existing chunk, just return "No chunks"

                    Example:
                    Input:
                        - Proposition: "Greg really likes hamburgers"
                        - Current Chunks:
                            - Chunk ID: 2n4l3d
                            - Chunk Name: Places in San Francisco
                            - Chunk Summary: Overview of the things to do with San Francisco Places

                            - Chunk ID: 93833k
                            - Chunk Name: Food Greg likes
                            - Chunk Summary: Lists of the food and dishes that Greg likes
                    Output: 93833k
                    """,
                ),
                ("user", "Current Chunks:\n--Start of current chunks--\n{current_chunk_outline}\n--End of current chunks--"),
                ("user", "Determine if the following statement should belong to one of the chunks outlined:\n{proposition}"),
            ]
        )

        runnable = PROMPT | self.llm

        chunk_found = runnable.invoke({
            "proposition": proposition,
            "current_chunk_outline": current_chunk_outline
        }).content

        # Pydantic data class
        class ChunkID(BaseModel):
            """Extracting the chunk id"""
            chunk_id: Optional[str]

        # Extraction to catch-all LLM responses. This is a bandaid
        extraction_chain = create_extraction_chain_pydantic(pydantic_schema=ChunkID, llm=self.llm)
        extraction_found = extraction_chain.run(chunk_found)
        if extraction_found:
            chunk_found = extraction_found[0].chunk_id

        # If you got a response that isn't the chunk id limit, chances are it's a bad response or it found nothing
        # So return nothing
        if len(chunk_found) != self.id_truncate_limit:
            return None

        return chunk_found

    def get_chunks(self, get_type='dict'):
        """
        This function returns the chunks in the format specified by the 'get_type' parameter.
        If 'get_type' is 'dict', it returns the chunks as a dictionary.
        If 'get_type' is 'list_of_strings', it returns the chunks as a list of strings, where each string is a proposition in the chunk.
        """
        if get_type == 'dict':
            return self.chunks
        if get_type == 'list_of_strings':
            chunks = []
            for chunk_id, chunk in self.chunks.items():
                chunks.append(" ".join([x for x in chunk['propositions']]))
            return chunks

    def pretty_print_chunks(self):
        print (f"\nYou have {len(self.chunks)} chunks\n")
        for chunk_id, chunk in self.chunks.items():
            print(f"Chunk #{chunk['chunk_index']}")
            print(f"Chunk ID: {chunk_id}")
            print(f"Summary: {chunk['summary']}")
            print(f"Propositions:")
            for prop in chunk['propositions']:
                print(f"    -{prop}")
            print("\n\n")

    def pretty_print_chunk_outline(self):
        print ("Chunk Outline\n")
        print(self.get_chunk_outline())

ac = AgenticChunker()

## Comment and uncomment the propositions to your hearts content
propositions = [
    "The only way to be successful is to get up at the crack of dawn – or so the story goes.",
    "But early rising productivity is not a one-size-fits-all situation, Bryan Lufkin finds.",
    "Being successful means waking up early – or so we’re constantly told."
]

ac.add_propositions(propositions)
ac.pretty_print_chunks()
ac.pretty_print_chunk_outline()
print (ac.get_chunks(get_type='list_of_strings'))

/home/dungca/anaconda3/envs/llm_retrieval_rag/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


AttributeError: module 'IPython.utils.traitlets' has no attribute 'Unicode'